In [1]:
import torch
from torch_cka import CKA
import rootutils
from omegaconf import OmegaConf
from torchvision.transforms import RandAugment
from hydra import compose, initialize

ROOT = rootutils.setup_root("/home/phisch/multimodal", indicator=".project-root", pythonpath=True)

from src.model.utils import get_model_and_processor
from src.model.model_module import LitMML
from src.data.data_module import MyDataModule

In [2]:
with initialize(config_path="configs"):
    cfg = compose(config_name="test", overrides=["checkpoints=full_dataset"])

/tmp/ipykernel_2653382/2639836681.py:1: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path="configs"):


In [3]:
model_config = cfg.model
ckpt_path_clip = "/home/data/bhavin/ckpts/yh1adr3g/last.ckpt"
ckpt_path_clip_simclr_itm = "/home/data/bhavin/ckpts/9nvg456i/last.ckpt"
ckpt_path_clip_itm = "/home/data/bhavin/ckpts/93t3xgrr/last.ckpt"

In [4]:
model, processor = get_model_and_processor(model_config)

lit_model = LitMML.load_from_checkpoint(ckpt_path_clip, 
                                        model=model, 
                                        processor=processor,
                                        )
model_clip = lit_model.model

lit_model = LitMML.load_from_checkpoint(ckpt_path_clip_itm, 
                                        model=model, 
                                        processor=processor,
                                        )
model_clip_itm = lit_model.model

/home/phisch/venv_py3.8/py3.8/lib/python3.8/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.2.3, which is newer than your current Lightning version: v2.2.0.post0


In [64]:
for i, (name, layer) in enumerate(model_clip.named_modules()):
    print(i, name)

0 
1 vision_model
2 vision_model.embeddings
3 vision_model.embeddings.patch_embeddings
4 vision_model.embeddings.patch_embeddings.projection
5 vision_model.embeddings.dropout
6 vision_model.encoder
7 vision_model.encoder.layer
8 vision_model.encoder.layer.0
9 vision_model.encoder.layer.0.attention
10 vision_model.encoder.layer.0.attention.attention
11 vision_model.encoder.layer.0.attention.attention.query
12 vision_model.encoder.layer.0.attention.attention.key
13 vision_model.encoder.layer.0.attention.attention.value
14 vision_model.encoder.layer.0.attention.attention.dropout
15 vision_model.encoder.layer.0.attention.output
16 vision_model.encoder.layer.0.attention.output.dense
17 vision_model.encoder.layer.0.attention.output.dropout
18 vision_model.encoder.layer.0.intermediate
19 vision_model.encoder.layer.0.intermediate.dense
20 vision_model.encoder.layer.0.intermediate.intermediate_act_fn
21 vision_model.encoder.layer.0.output
22 vision_model.encoder.layer.0.output.dense
23 vision_m

In [5]:
layers = [f"vision_model.encoder.layer.{i}" for i in range(12)] + ["vision_model.pooler", "visual_projection"]

In [6]:
cka = CKA(model_clip, model_clip_itm,
    model1_name = "CLIP", model2_name = "CLIP + ITM",
    model1_layers=layers, model2_layers=layers,
    device="cuda:3")
cka.model1 = lambda x: model_clip.get_image_features(pixel_values=x)
cka.model2 = lambda x: model_clip_itm.get_image_features(pixel_values=x)

In [7]:
data_config = cfg.data
data_module = MyDataModule(data_config, processor, augmentation=RandAugment(), num_views=2)
test_dataloaders = data_module.get_test_dataloaders()

In [8]:
cifar10_dataloader = test_dataloaders["CIFAR10"]["test"]

In [24]:
del batch
torch.cuda.empty_cache()

In [9]:
batch = next(iter(cifar10_dataloader))

In [13]:
batch[1]

tensor([3, 8, 8, 0, 6, 6, 1, 6, 3, 1, 0, 9, 5, 7, 9, 8, 5, 7, 8, 6, 7, 0, 4, 9,
        5, 2, 4, 0, 9, 6, 6, 5, 4, 5, 9, 2, 4, 1, 9, 5, 4, 6, 5, 6, 0, 9, 3, 9,
        7, 6, 9, 8, 0, 3, 8, 8, 7, 7, 4, 6, 7, 3, 6, 3, 6, 2, 1, 2, 3, 7, 2, 6,
        8, 8, 0, 2, 9, 3, 3, 8, 8, 1, 1, 7, 2, 5, 2, 7, 8, 9, 0, 3, 8, 6, 4, 6,
        6, 0, 0, 7, 4, 5, 6, 3, 1, 1, 3, 6, 8, 7, 4, 0, 6, 2, 1, 3, 0, 4, 2, 7,
        8, 3, 1, 2, 8, 0, 8, 3])

In [10]:
feats = model_clip_simclr_itm.get_image_features(pixel_values=batch[0].cuda("cuda:3"))

In [12]:
feats.shape

torch.Size([128, 512])

In [ ]:
cka.compare(cifar10_dataloader)

/home/phisch/venv_py3.8/py3.8/lib/python3.8/site-packages/torch_cka/cka.py:145: UserWarning: Dataloader for Model 2 is not given. Using the same dataloader for both models.
  warn("Dataloader for Model 2 is not given. Using the same dataloader for both models.")


| Comparing features |:   0%|          | 0/79 [00:00<?, ?it/s]ERROR:tornado.general:SEND Error: Host unreachable
| Comparing features |:   0%|          | 0/79 [07:08<?, ?it/s]
